## Heat Transfer through pipe

In [25]:
#effective Heat transfer coefficient
import numpy as np
from scipy.optimize import root
from scipy.optimize import fsolve

#Reynold's number calculations
def Re (mdot,D,mew_gas):
    _mdot=mdot
    _D=D
    _mew_gas=mew_gas
    _Re= 4*_mdot/(np.pi*_D*_mew_gas)
    return _Re

#for inside of pipe
def fric_factor(e,D,Re):
    _e=e
    _D=D
    _Re=Re
    def equation(f):
        return ((1/f**0.5)+2.0*np.log(_e/_D/3.7+2.51/(_Re*f**0.5)))
    _f= root(equation, 0.4, method='hybr')
    print ('friction factor=',_f, "error=",equation(_f))
    
    return (_f)

def Nu_in(f, Re, Pr):
    _f=f
    _Re=Re
    _Pr=Pr
    Nu= ((_f/8 * (_Re-1000)*_Pr)/(1+12.7*(_f/8)**0.5*(_Pr**(2/3)-1))) #Gnielinski correlation, most eneralised for both conditions
    return Nu

def h(Nu,D,k_fluid):
    _Nu=Nu
    _D=D
    _k_fluid=k_fluid
    h = Nu*k_fluid/D
    return h

def main(k_pipe,k_gas,k_air, Pr_gas, Pr_air,mdot,T0, T_amb, L, D, thick, e, mew_gas, kin_visc_air):
    _k_pipe=k_pipe
    _k_gas=k_gas
    _k_air=k_air
    _Pr_gas=Pr_gas
    _Pr_air=Pr_air
    _mdot=mdot
    _T0=T0
    _T_amb=T_amb
    _L=L
    _D=D
    _e=e
    _thick=thick
    _mew_gas=mew_gas
    _kin_visc_air=kin_visc_air
    _D_ext=D+2*_thick
    g=9.81
    #for inside the pipe
    _Re= Re(_mdot,_D,_mew_gas)
    _f=fric_factor(_e,_D,_Re)
    _Nu=Nu_in(_f,_Re,_Pr_gas)
    h_in= h(_Nu,_D,_k_gas)
    
    #for the pipe thickness
    #h_thick= _k_pipe/_thick
    
    #for the outside of pipe
    #_Gr= Gr(Ts,_T_amb, _D_ext,_kin_visc_air)
    #_Nu_out=Nu_out(_Gr,_Pr_air)
    #h_out= h(_Nu_out,_D_ext,_k_air)
    
    def sur_temp_equation(Ts):  #equation derived by considered the pipe as lump i.e.constant temperature throughout
        return (((0.60+ 0.387*(((g*(1/(Ts+_T_amb))*(Ts-_T_amb)*_D_ext**3/_kin_visc_air**2))*_Pr_air)**(1/6)/(1+(0.559/_Pr_air)**(9/16))**(8/27))**2*_k_air/_D_ext)*(Ts-_T_amb)-h_in*(_T0-Ts))
    _T_s=root(surf_temp_equation, 280)
    print ('Surface temp=',_T_s,'error=',sur_temp_equation(_T_s))
    
    return _T_s

main(52,25,25, 0.72, 0.71,1.41,285, 305, 1, 0.0508, 4, 260, 27*1e-6, 18*1e-6)

TypeError: unsupported operand type(s) for ** or pow(): 'OptimizeResult' and 'float'

In [27]:
#checking friction fator function

import numpy as np
from scipy.optimize import root
def fric_factor(e,D,Re):
    _e=e
    _D=D
    _Re=Re
    def equation(f):
        return ((1/f**0.5)+2.0*np.log(_e/_D/3.7+2.51/(_Re*f**0.5)))
    _f=root(equation, 0.004)
    print ('friction factor=',_f, "error=",equation(_f))

_friction = fric_factor(260,0.05,100000)

TypeError: unsupported operand type(s) for ** or pow(): 'OptimizeResult' and 'float'

In [6]:
_e=260
_D=0.50
_Re=100000
def equation(f):
    return ((1/f**0.5)+2.0*np.log(_e/_D/3.7+2.51/(_Re*f**0.5)))
error_frict= equation(0.005)
print (error_frict)

24.03313265903722


In [23]:
#Checking Reynolds number function

def Re (mdot,D,mew_gas):
    _mdot=mdot
    _D=D
    _mew_gas=mew_gas
    _Re= 4*_mdot/(np.pi*_D*_mew_gas)
    return _Re
print(Re(1.4,0.025,27*1e-6))

2640793.1298210784


In [35]:
#Grashof function for outer coefficient of heat transfer
def Grashof(Ts,T_amb,D_ext,kin_visc_air):
    g=9.81
    Tavg= (Ts+T_amb)/2
    beta= 1/Tavg
    Gr= g*beta*(Ts-T_amb)*D_ext**3/kin_visc_air**2
    return Gr

print (Grashof(250,295,0.04,18*1e-6))

#Grashof number less than 10**8 shows its laminar which should be the case for Naural convection


-320000.00000000006


In [36]:
def Nu_out(Gr, Pr):
    _Gr=Gr
    _Pr=Pr
    Ra=_Gr*_Pr
    Nu_avg= (0.60+ 0.387*Ra**(1/6)/(1+(0.559/_Pr)**(9/16))**(8/27))**2
    return Nu_avg

print(Nu_out(320000,0.7))

9.616001082499588


In [38]:
def h(Nu,D,k_fluid):
    _Nu=Nu
    _D=D
    _k_fluid=k_fluid
    h = Nu*k_fluid/D
    return h

print (h(9.61,0.040,0.02))

4.805


In [42]:

_T_amb=295
_kin_visc_air=27*1e-6
_Pr_air=0.7
_k_air= 0.02
_D_ext= 0.04
_T0=240
h_in=240
g=9.81
def sur_temp_equation(Ts):
    return (((0.60+ 0.387*(((g*(1/(Ts+_T_amb))*(Ts-_T_amb)*_D_ext**3/_kin_visc_air**2))*_Pr_air)**(1/6)/(1+(0.559/_Pr_air)**(9/16))**(8/27))**2*_k_air/_D_ext)*(Ts-_T_amb)-h_in*(_T0-Ts))

print (sur_temp_equation(250))

(2303.7654377812123-100.09986283915462j)
